In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [2]:
#Training Set
train_dataset=datasets.MNIST(root='./',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
#Testing Set
test_dataset=datasets.MNIST(root='./',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)


In [3]:
#batch size
batch_size=64
#training set loader
train_loader=DataLoader(dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=True)
#Testing set loader
test_loader=DataLoader(dataset=test_dataset,
                       batch_size=batch_size,
                       shuffle=True)
#print dataset
for i,data in enumerate(train_loader):
    inputs,labels=data
    print(inputs.shape)
    print(labels.shape)
    break
    
print(labels)
print(len(train_loader))
print(len(test_loader))

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([4, 5, 0, 3, 5, 1, 7, 6, 4, 3, 8, 8, 0, 5, 1, 4, 3, 4, 7, 7, 4, 3, 9, 4,
        8, 4, 9, 5, 3, 4, 8, 6, 7, 6, 4, 5, 6, 9, 1, 4, 3, 4, 5, 5, 1, 4, 6, 1,
        4, 5, 9, 7, 3, 5, 0, 7, 5, 7, 8, 8, 7, 2, 5, 6])
938
157


In [4]:
#define network algorithm
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,10)
        self.softmax=nn.Softmax(dim=1)
    
    def forward(self,x):
        x=x.view(x.size()[0],-1) #change ([64,1,28,28]) to (64,784),-1 is auto match data
        x=self.fc1(x)
        x=self.softmax(x)
        return x
# define module
model=Net()
#define mse_loss function
mse_loss=nn.MSELoss()
#define optimizer with learning rate
optimizer=optim.SGD(model.parameters(),lr=0.01)

In [5]:
#define module training
def train():
    for i, data in enumerate(train_loader):
        #obtain a batch of inputs and labels
        inputs, labels=data
        #obtain the module predict result (64,10)
        out =model(inputs)
        # change label to one hot code
        labels=labels.reshape(-1,1) #change dimension (64)->(64,1)
        one_hot=torch.zeros(inputs.shape[0],10).scatter(1,labels,1)
        #calculate loss value(needs have same shape with mse_loss)
        loss=mse_loss(out,one_hot)
        #initial gradient value
        optimizer.zero_grad()
        #calculate gradient 
        loss.backward()
        #renew weight
        optimizer.step()
#define module testing
def test():
    correct=0
    for i,data in enumerate(test_loader):
        #obtain a batch of inputs and labels
        inputs,labels=data
        #obtain the module predict result (64,10)
        out=model(inputs)
        #obtain max value and it position
        _, predicted=torch.max(out,1)
        # predict correct amount
        correct += (predicted==labels).sum()
        print("Test accuracy:{0}".format(correct.item()/len(test_dataset)))

In [6]:
for epoch in range(10):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test accuracy:0.0036
Test accuracy:0.0065
Test accuracy:0.0095
Test accuracy:0.0126
Test accuracy:0.0155
Test accuracy:0.0188
Test accuracy:0.0225
Test accuracy:0.0259
Test accuracy:0.029
Test accuracy:0.0317
Test accuracy:0.0353
Test accuracy:0.039
Test accuracy:0.042
Test accuracy:0.0461
Test accuracy:0.0497
Test accuracy:0.0533
Test accuracy:0.0567
Test accuracy:0.0605
Test accuracy:0.0642
Test accuracy:0.0674
Test accuracy:0.0703
Test accuracy:0.074
Test accuracy:0.0772
Test accuracy:0.0803
Test accuracy:0.0841
Test accuracy:0.0877
Test accuracy:0.0904
Test accuracy:0.0934
Test accuracy:0.097
Test accuracy:0.1005
Test accuracy:0.1044
Test accuracy:0.108
Test accuracy:0.112
Test accuracy:0.115
Test accuracy:0.1185
Test accuracy:0.1218
Test accuracy:0.1254
Test accuracy:0.1286
Test accuracy:0.1325
Test accuracy:0.1356
Test accuracy:0.1391
Test accuracy:0.1417
Test accuracy:0.1451
Test accuracy:0.1482
Test accuracy:0.1523
Test accuracy:0.1563
Test accuracy:0.1593
Test accurac

In [9]:
torch.save(model.state_dict(),'module/my_module.pb')